# Tube Turbulent with 2 groupes IATE - Verification

# Description

The objective of this document is to verify that the 2 groups interfacial area equations work correctly without thermal effects and that the computation converges in a standard manner towards the correct solution.

In [ ]:
from trustutils import run 
import os
import math

run.TRUST_parameters()



# Definition of probes

In [ ]:
sondes = { }

n_seg_sonde = 40
r_tube = 0.019

tab_name_sondes = ["k",
                   "omega",
                   "vitesse_liquide_eau",
                   "vitesse_gaz_air_group1",
                   "vitesse_gaz_air_group2",
                   "alpha_gaz_air_group1",
                   "alpha_gaz_air_group2",
                   "pression",
                   "dp",
                   "lift",
                   "disp",
                   "vorticite",
                   "d_bulles",
                   "interfacial_area"
                  ]

x_0 = r_tube/(2*n_seg_sonde)*math.cos(math.pi*.5/360)
y_0 = r_tube/(2*n_seg_sonde)*math.sin(math.pi*.5/360)

x_wall = r_tube*(2*n_seg_sonde-1)/(2*n_seg_sonde)*math.cos(math.pi*.5/360)
y_wall = r_tube*(2*n_seg_sonde-1)/(2*n_seg_sonde)*math.sin(math.pi*.5/360)

z_sonde = 0.38

str_sonde = " "
for name_sonde in tab_name_sondes:
    str_sonde += f" {name_sonde} {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x_0} {y_0} {z_sonde} {x_wall} {y_wall} {z_sonde}  \n"

sondes["PolyMAC"] = str_sonde


# Preparation and launching of the computation

In [ ]:
force_recalculation = True 
number_of_partitions = 4

# Mesh
med_file = {  "PolyMAC" : "1_tube_analytique.med" }
# use smaller mesh for NR test
if run.isExtractingNR():
    med_file = {  "PolyMAC" : "1_tube_analytique_grossier.med" }

name_mesh  = { "PolyMAC" : "read_med { domain dom mesh mesh file " + med_file["PolyMAC"] + " }" }

schema = { "PolyMAC" : " PolyMAC_P0  dis option_PolyMAC { traitement_axi } " }
dimension = {  "PolyMAC" : 3}
bidim_axi = {  "PolyMAC" : " "}
nrad = {   "PolyMAC" : 40}
solveur = { "ICE" : ""}
paroi = {   "PolyMAC" : " paroi "}
paroi_adiabatique = { "PolyMAC" : " paroi_adiabatique "}

# Numerical parameters
facsec  = 0.5
tmax    = 1
nb_pas_dt_max = '20000'
seuil_statio  = '1.e-3'

#Physical quantities
air_properties_1 =   "     gaz_air_group1 Fluide_Incompressible { mu champ_uniforme 1 1.e-5 rho champ_uniforme 1  1.   lambda Champ_Uniforme  1 0.023 Cp Champ_Uniforme 1 1006   beta_th Champ_Uniforme 1 0 } "
air_properties_2 =   "     gaz_air_group2 Fluide_Incompressible { mu champ_uniforme 1 1.e-5 rho champ_uniforme 1  1.   lambda Champ_Uniforme  1 0.023 Cp Champ_Uniforme 1 1006   beta_th Champ_Uniforme 1 0 } "
water_properties = " liquide_eau Fluide_Incompressible { mu champ_uniforme 1 1.e-3 rho champ_uniforme 1  1.e3 lambda Champ_Uniforme  1 0.604 Cp Champ_Uniforme 1 75.366 beta_th Champ_Uniforme 1 0 } "
interface =        "interface_eau_air interface_sigma_constant  { tension_superficielle 0.0728 }"

# Steady state target
u_0 = 20000.*1.e-6/0.038 #Re 20000
alpha_cible = 0.1

#Turbulent quantities
D_h = 0.038
CI_k  = 0.01*u_0**2
CI_om = CI_k**0.5*0.09**0.5/0.42/D_h/0.1
        
#Two-phase quantities
Cd = 2.
frottement_interfacial = " bulles_constant { coeff_derive " + f"{Cd}" + " } " 
Cl = 0.03
portance_interfaciale = " constante { Cl " + f"{Cl}" + " } " 
D_td_star = 0.03
dispersion_bulles = " constante {   D_td_star " + f"{D_td_star}" +" } " 
db1 = 0.0025
db2 = 0.012
beta_portance = "1"
beta_disp     = "1"
Cw1 = -.1
Cw2 = .147

# Guess of inlet conditions for target
u_r1 = math.sqrt(4.*db1/(3*Cd)*9.81*(1000.-1.)/1000.*(1-2*alpha_cible))
u_r2 = math.sqrt(4.*db2/(3*Cd)*9.81*(1000.-1.)/1000.*(1-2*alpha_cible))
alpha_entree1 = alpha_cible *(u_0+u_r1)/u_0
alpha_entree2 = alpha_cible *(u_0+u_r2)/u_0
ai1 = 6*alpha_entree1/db1
ai2 = 6*alpha_entree2/db2

# Building computation
build_dir = run.BUILD_DIRECTORY

if force_recalculation or not os.path.exists(f'{build_dir}'):
    print('Recalculation running..')
    run.reset()

    for s in schema.keys() :
        for solv in solveur.keys() :
            name = f"{s}_{solv}"

            substitutions_dict = {"dimension" : dimension[s],
                                    "bidim_axi" : bidim_axi[s],
                                    "name_mesh" : f'{name_mesh[s]}' ,
                                    "schema" : schema[s],

                                    "tmax": str(tmax),                                  
                                    "facsec" : str(facsec) ,
                                    "nb_pas_dt_max" : nb_pas_dt_max ,
                                    "seuil_statio" : seuil_statio ,
                                    "solveur" : solv ,
                                    "facsec_diffusion_for_sets" : solveur[solv] ,
                                                
                                    "carrying_phase" : water_properties ,
                                    "dispersed_phase_1" : air_properties_1 ,
                                     "dispersed_phase_2" : air_properties_2 ,
                                    "interface" : interface,
                                                
                                    "diametre_bulles" : str(db1),
                                    "u_0"  : f"{3*dimension[s]} {''.join([ ' 0 ' for i in range(3*(dimension[s]-1))])} {u_0} {u_0}*1.1142 {u_0}*1.1142 " ,
                                    "grav"   : f"{3*dimension[s]} {''.join([ ' 0 ' for i in range(3*(dimension[s]-1))])} -9.81 -9.81 -9.81 " ,
                                    "alpha_l0" : str(1-alpha_entree1-alpha_entree2) ,
                                    "alpha_v01" : str(alpha_entree1) ,
                                    "alpha_v02" : str(alpha_entree2) ,
                                    "frottement_interfacial" : frottement_interfacial,
                                    "portance_interfaciale" : portance_interfaciale,
                                    "dispersion_bulles" : dispersion_bulles,
                                    "beta_portance" : beta_portance,
                                    "beta_disp" : beta_disp,
                                                
                                    "CI_diss": f"{CI_om}" ,
                                    "CI_k": f"{CI_k}" ,
                                    "paroi_adiabatique" : paroi_adiabatique[s],
                                    "paroi" : paroi[s],

                                    "ci_ai1" : str(ai1),
                                    "ci_ai2" : str(ai2),

                                    "sondes" : sondes[s],
                                                
                                }

            tc = run.addCaseFromTemplate("jdd.data",
                                         targetDirectory=f"{name}",
                                         dic=substitutions_dict,
                                         nbProcs=number_of_partitions)

            if (med_file[s]!=""):
                os.system(f'cp {build_dir}/{med_file[s]} {build_dir}/{name}/{med_file[s]}')
            if number_of_partitions > 1:
                tc.partition()

    run.printCases()
    run.runCases()
    display(run.tablePerf())

else:
    print('No recalculation, reusing old results!')

# Loading results

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
matplotlib.rcParams['text.usetex'] = True

from trustutils.jupyter import plot

plt.rcParams["font.weight"] = "bold"

############# results : sim[config][debit air][raf][position]
#############         columns : ['y/h'] ['vxl'] ['vyl'] ['vxg'] ['vyg'] ['k'] ['alp'] ['DB']
#############                   ["diff"] ["conv"] ["press"] ["grav"] ["drag"] ["lift"] ["disp"] ["lub"]
#############               nb : end column name with lx for forces
############# sim name : tab_name [config][debit air][raf]
############# sim label: tab_label[config][debit air][raf]
############# convergence :  dt_ev[config][debit air][raf]
#############         columns : ['time']['dt']['facsec']['residu']['dt_stab']['vitl']['vitg']['alphal']['alphag']['Tl']['Tg']['diss']['k'] 

sim      = {  }
dt_ev    = {  }
par = ""
if number_of_partitions > 1 : par = "PAR_"

for s in schema.keys() :
    for solv in solveur.keys() :
        name = f"{s}_{solv}"

        dt_ev[name] = pd.read_csv(f"{build_dir}/{name}/{par}jdd.dt_ev", sep="\t")
        dt_ev[name] = dt_ev[name].iloc[:,:14]
        dt_ev[name].columns = ['time', 'dt', 'facsec', 'residu', 'dt_stab', 'vitl', 'vitg1','vitg2', 'alphal', 'alphag1', 'alphag2', 'Tl', 'Tg1','Tg2']

        sim[name] = pd.DataFrame()
        sim[name]['r+']  = np.linspace(1.0/(2*n_seg_sonde), 1 - 1.0/(2*n_seg_sonde), n_seg_sonde)
        sim[name]['vxl'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU.son"))[1::dimension[s],-1]
        sim[name]['vzl'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_EAU.son"))[dimension[s]::dimension[s],-1]
        sim[name]['vxg1'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1.son"))[1::dimension[s],-1]
        sim[name]['vzg1'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP1.son"))[dimension[s]::dimension[s],-1]
        sim[name]['vxg2'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2.son"))[1::dimension[s],-1]
        sim[name]['vzg2'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_AIR_GROUP2.son"))[dimension[s]::dimension[s],-1]
        sim[name]['k']   = np.array(plot.loadText(f"{name}/{par}jdd_K.son"))[1::,-1]
        sim[name]['alp1'] = np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP1.son"))[1::,-1]
        sim[name]['alp2'] = np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_AIR_GROUP2.son"))[1::,-1]
        sim[name]['p']   = np.array(plot.loadText(f"{name}/{par}jdd_PRESSION.son"))[1::,-1]
        sim[name]['dp']  = np.array(plot.loadText(f"{name}/{par}jdd_DP.son"))[1::,-1]
        sim[name]['db1']  = np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES.son"))[1::,-1][1::3]
        sim[name]['db2']  = np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES.son"))[1::,-1][2::3]
        sim[name]['ai1']  = np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA.son"))[1::,-1][1::3]
        sim[name]['ai2']  = np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA.son"))[1::,-1][2::3]




# Convergence plots

In [ ]:
matplotlib.rcParams.update({'font.size': 14})

color_conf = { "VDF_ICE" : "red", "PolyMAC_ICE" : "magenta", "VDF_SETS" : "orange", "PolyMAC_SETS" : "purple"  }

fig = plt.figure(figsize = (16, 4))

name_phys = [ ["vitg1"],["vitg2"], [ "alphag1"],  [ "alphag2"]]

axs = fig.subplots(1, len(name_phys) )#, sharex=True, sharey=True)

for l in range(len(name_phys)): #phys
    for r in dt_ev.keys() :
        for lp in range(len(name_phys[l])) : #physique de la phase si multiphase
            axs[l].plot(dt_ev[r]["time"][::], dt_ev[r][name_phys[l][lp]][::], "-", label = r.replace("_", " "), color = color_conf[r])

        axs[l].set_xlim(0,tmax)
        axs[l].set_ylim(float(seuil_statio)/100, 1e4)
        axs[l].set_yscale("log")
        axs[l].set_title("convergence "+name_phys[l][0] )
        axs[l].set_xlabel('time')
        axs[l].set_ylabel("convergence " + name_phys[l][0])
        axs[l].legend(prop={'size': 10})

plt.tight_layout()

# Void fraction and velocities plots

In [ ]:
# Analytical guess

l_sim = len(sim["PolyMAC_ICE"]["r+"] )
rp  = sim["PolyMAC_ICE"]["r+"] 
r   = rp * D_h/2.
Vl  = np.array(sim["PolyMAC_ICE"]["vzl"])   
DB1  = np.array(sim["PolyMAC_ICE"]["db1"]) 
DB2  = np.array(sim["PolyMAC_ICE"]["db2"]) 
Ur1  = np.zeros(l_sim) + np.sqrt(4.*DB1/(3*Cd)*9.81*(1000.-1.)/1000.*(1-alpha_entree1-alpha_entree2)) 
Ur2  = np.zeros(l_sim) + np.sqrt(4.*DB2/(3*Cd)*9.81*(1000.-1.)/1000.*(1-alpha_entree1-alpha_entree2)) 

alp1 = Vl/(Vl+Ur1)*alpha_entree1
alp2 = Vl/(Vl+Ur2)*alpha_entree2

# Colonnes : Titre, alpha, d, U (contient U_l et U_g), u'l
  
color_group1 = "darkorchid"
color_group2 = "royalblue"

marker_conf =  { "PolyMAC_ICE" : "--" }

plt.figure(figsize = (12,6))

plt.subplot(221)
plt.plot(r*1000, alp1, "^", color = color_group1,  label = "1D solution")
plt.xlim(0,20)
for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        time_loc = str(float(os.read(f, 200).split()[3])/60)[:5]
        plt.plot(np.array(sim[s]["r+"])*0.019*1000, sim[s]['alp1'], marker_conf[s], linewidth = 3, label = f"Group1 {s.replace('_',' ')} ", color = color_group1)

plt.legend(fontsize = 8)
plt.xlabel("Radius (mm)")
plt.ylabel("Void fraction")
plt.title("Comparison between 1D solution and Group 1 results")
    
plt.subplot(222)

plt.plot(r*1000, alp2, "^", color = color_group2,  label = "1D solution")

for s in dt_ev.keys():
        plt.plot(np.array(sim[s]["r+"])*0.019*1000, sim[s]['alp2'], marker_conf[s], linewidth = 3, label = f"Group2 {s.replace('_',' ')}", color = color_group2)

plt.legend(fontsize = 8)
plt.xlabel("Radius (mm)")
plt.ylabel("Void fractions")
plt.title("Comparison between 1D solution and Group 2 results")

plt.subplot(223)
plt.plot(r*1000, Ur1+Vl, "^", color = color_group1,  label = "1D solution")
plt.xlim(0,20)
for s in dt_ev.keys():
    if (s != "VDF_SETS"):
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        time_loc = str(float(os.read(f, 200).split()[3])/60)[:5]
        plt.plot(np.array(sim[s]["r+"])*0.019*1000, sim[s]['vzg1'], marker_conf[s], linewidth = 3, label = f"Group1 {s.replace('_',' ')}", color = color_group1)

plt.legend(fontsize = 8)
plt.xlabel("Radius (mm)")
plt.ylabel("Gas velocity")

plt.subplot(224)
plt.plot(r*1000, Ur2+Vl, "^", color = color_group2,  label = "1D solution")
plt.xlim(0,20)
for s in dt_ev.keys():
        plt.plot(np.array(sim[s]["r+"])*0.019*1000, sim[s]['vzg2'], marker_conf[s], linewidth = 3, label = f"Group2 {s.replace('_',' ')}", color = color_group2)

plt.legend(fontsize = 8)
plt.xlabel("Radius (mm)")
plt.ylabel("Gas velocity")


plt.tight_layout()

# Bubble diameter profiles

In [ ]:
# Colonnes : Titre, alpha, d, U (contient U_l et U_g), u'l

    
color_group1 = "darkorchid"
color_group2 = "royalblue"
marker_conf =  { "VDF_ICE" : "--", "VDF_SETS" : "-", "PolyMAC_ICE" : "--" , "PolyMAC_SETS" : "-"}

plt.figure(figsize = (12,6))

plt.subplot(121)
plt.xlim(0,20)
for s in dt_ev.keys():
    if (s != "VDF_SETS"):
        plt.plot(np.array(sim[s]["r+"])*0.019*1000, sim[s]['db1']*1000, marker_conf[s], linewidth = 3, label = f"Group1 {s.replace('_',' ')} ", color = color_group1)

plt.legend(fontsize = 8)
plt.xlabel("Radius (mm)")
plt.ylabel("Bubble diameter group 1 (mm)")
plt.title("Group 1 bubble diameter")
    
plt.subplot(122)
plt.xlim(0,20)
#plt.ylim(0,1.5)
for s in dt_ev.keys():
    if (s != "VDF_SETS"):
        plt.plot(np.array(sim[s]["r+"])*0.019*1000, sim[s]['db2']*1000, marker_conf[s], linewidth = 3, label = f"Group2 {s.replace('_',' ')} ", color = color_group2)

plt.legend(fontsize = 8)
plt.xlabel("Radius (mm)")
plt.ylabel("Bubble diameter group 2 (mm)")
plt.title("Group 2 bubble diameter ")

plt.tight_layout()

# Interfacial area concentration profiles

In [ ]:

# Colonnes : Titre, alpha, d, U (contient U_l et U_g), u'l
    
color_group1 = "darkorchid"
color_group2 = "royalblue"
marker_conf =  { "VDF_ICE" : "--", "VDF_SETS" : "-", "PolyMAC_ICE" : "--" , "PolyMAC_SETS" : "-"}

plt.figure(figsize = (12,6))

plt.subplot(121)
plt.xlim(0,20)
for s in dt_ev.keys():
    if (s != "VDF_SETS"):
        plt.plot(np.array(sim[s]["r+"])*0.019*1000, sim[s]['ai1'], marker_conf[s], linewidth = 3, label = f"Group1 {s.replace('_',' ')} ", color = color_group1)

plt.legend(fontsize = 8)
plt.xlabel("Radius (mm)")
plt.ylabel(r"Interfacial area concentration group 1 ($m^{-1}$)")
plt.title("Interfacial area concentration")
    
plt.subplot(122)
plt.xlim(0,20)
for s in dt_ev.keys():
    if (s != "VDF_SETS"):
        plt.plot(np.array(sim[s]["r+"])*0.019*1000, sim[s]['ai2'], marker_conf[s], linewidth = 3, label = f"Group2 {s.replace('_',' ')} ", color = color_group2)

plt.legend(fontsize = 8)
plt.xlabel("Radius (mm)")
plt.ylabel(r"Interfacial area concentration group 2 ($m^{-1}$)")
plt.title("Interfacial area concentration")

plt.tight_layout()